# Forecasting Pipeline

# Global LGBM + RF Forecaster

In [ ]:
import numpy as np
import pandas as pd
from datetime import timedelta
from sqlalchemy import text
import hashlib
from tqdm.notebook import tqdm
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error
import optuna
from database.db_utils import get_db_connection
from sklearn.ensemble import RandomForestRegressor
from forecasting.data_preprocessing import preprocess_data, create_lagged_features
from sklearn.preprocessing import LabelEncoder


try:
    import lightgbm as lgb
    _USE_LGBM = True
except Exception:
    from xgboost import XGBRegressor
    _USE_LGBM = False


/Users/panayotispapoutsis/Documents/StablecoinPoolOpt/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Controls
HIST_DAYS_PANEL   = 150
MIN_ROWS_PANEL    = 400

In [3]:
def get_filtered_pool_ids_readonly(limit=None):
    # Your function in fp.get_filtered_pool_ids() uses the same SQL; this read-only helper
    # avoids accidentally importing the writer in main().
    engine = get_db_connection()
    sql = """
    SELECT DISTINCT pool_id
    FROM pool_daily_metrics
    WHERE is_filtered_out = FALSE
    """
    with engine.connect() as conn:
        df = pd.read_sql(sql, conn)
    ids = df["pool_id"].tolist()
    return ids[:limit] if limit else ids

In [4]:
def _count_actual_history(panel_df: pd.DataFrame, pid: str, asof_norm: pd.Timestamp) -> int:
    return int(
        panel_df.loc[
            (panel_df['pool_id'] == pid) &
            (panel_df['date'] <= asof_norm) &
            (panel_df['actual_apy'].notna())
        ].shape[0]
    )


In [5]:
def fetch_panel_history(asof: pd.Timestamp, pool_ids: list, days:int=HIST_DAYS_PANEL) -> pd.DataFrame:
    """
    Read-only: fetch last `days` of history up to `asof` for given pools.
    Returns tidy df with columns:
      date, pool_id, apy_7d, actual_apy, tvl_usd, eth_open, btc_open, gas_price_gwei, group_col
    """
    t = pd.Timestamp(asof)
    t = t.tz_localize('UTC') if t.tz is None else t.tz_convert('UTC')
    start = (t - pd.Timedelta(days=days)).normalize()

    engine = get_db_connection()
    q = """
        SELECT
            date,
            pool_id,
            rolling_apy_7d AS apy_7d,
            actual_apy,
            actual_tvl     AS tvl_usd,
            eth_open,
            btc_open,
            gas_price_gwei
        FROM pool_daily_metrics
        WHERE pool_id = ANY(:pool_ids)
          AND date >= :start_date
          AND date <= :asof_date
        ORDER BY date ASC
    """
    with engine.connect() as conn:
        df = pd.read_sql(
            text(q), conn,
            params={
                "pool_ids": pool_ids,
                "start_date": start.tz_convert('UTC').to_pydatetime(),
                "asof_date":  t.tz_convert('UTC').to_pydatetime()
            }
        )
    if df.empty:
        return df

    df['date'] = pd.to_datetime(df['date'], utc=True).dt.normalize()
    return df


In [6]:
def fetch_all_macro_daily(engine, start_date, end_date):
    """
    Loads ALL macro series between start_date and end_date,
    expands monthly to daily, fills missing days, and pivots to wide format.
    """

    # 1) Load raw macro data
    sql = """
        SELECT series_name, frequency, date, value
        FROM macroeconomic_data
        WHERE date BETWEEN :start AND :end
        ORDER BY date, series_name
    """
    df = pd.read_sql(
        text(sql),
        engine,
        params={"start": start_date, "end": end_date}
    )

    if df.empty:
        return pd.DataFrame()

    # 2) Normalize dates (remove timezone!)
    df["date"] = pd.to_datetime(df["date"]).dt.tz_localize('UTC')

    # 3) Create full daily grid
    all_days = pd.DataFrame({
        "date": pd.date_range(start_date, end_date, freq="D")
    })

    # 4) Pivot to wide format
    pivot = df.pivot(index="date", columns="series_name", values="value")

    # 5) Merge on full daily grid
    merged = all_days.merge(pivot, on="date", how="left")

    # 6) Forward fill to convert monthly → daily, fill weekends
    merged = merged.sort_values("date").ffill()
    merged = merged.sort_values("date").bfill()


    return merged

In [23]:
def add_neighbor_features(panel_df: pd.DataFrame, group_col: str = None) -> pd.DataFrame:
    """
    For each (date, pool), add group-level neighbor stats.

    Intended usage: predicting t+1 actual_apy using features at t.
    We provide both same-day (t) and past-only (t-1) variants to let you choose.

    Features (suffix _nbr):
      - group_tvl_sum_t_nbr                 (sum TVL at t)
      - group_apy_mean_t_nbr / median / std (based on apy_7d at t)
      - tvl_share_nbr                       (pool TVL share within its group at t)
      - apy_rank_nbr                        (normalized rank of apy_7d within its group at t, 0..1)
      - grp_ex_mean_t_nbr                   (group mean apy_7d at t excluding the pool)
      - grp_ex_mean_7d_nbr                  (7d rolling mean of grp_ex_mean_t by group)
      - *_lag1 counterparts for past-only neighbor stats (computed from t-1)

    Notes:
      - If `group_col` is None or missing, a single group 'ALL' is assumed (no leakage risk).
      - This function uses only columns: ['date','pool_id','apy_7d','tvl_usd', group_col].
        Make sure they exist upstream (we’ll create empties if missing to avoid crashes).
    """
    df = panel_df.copy()
    # Add Stability features
    # 1) Daily % change of APY per pool
    df['apy_pct_change'] = (
        df.groupby('pool_id')['actual_apy']
            .pct_change(fill_method=None)                 # (APY_t - APY_{t-1}) / APY_{t-1}
    )

    # 2) Absolute change = "instability" at each day
    df['apy_instability_abs'] = df['apy_pct_change'].abs()

    # 3) 3-day rolling instability *using only past days*:
    #    value at t = mean(|pct_change| at t-1, t-2, t-3)
    df['instability_3d'] = (
        df.groupby('pool_id')['apy_instability_abs']
            .transform(lambda s: s.shift(1).rolling(window=3, min_periods=1).mean())
    )

    # 4) Normalize to [0,1] per pool (nice as a feature)
    df['instability_3d_norm'] = (
        df.groupby('pool_id')['instability_3d']
            .transform(lambda s: (s - s.min()) / (s.max() - s.min() + 1e-9))
    )

    # 5) Number of days since first observation
    df["history_days"] = df.groupby("pool_id").cumcount()

    # 6) Instability thresholds
    mask = (df["history_days"] >= 7) & df["instability_3d"].notna()
    valid_instability = df.loc[mask, "instability_3d"]

    # Typical quantiles: 50% and 85% (same good practice as volatility)
    low_thr, high_thr = valid_instability.quantile([0.50, 0.85]).tolist()
    # Classify instability
    df[group_col] = df.apply(lambda x : classify_instability(x,low_thr,high_thr), axis=1)


    if "date" in df.columns:
        df["asof"] = pd.to_datetime(df["date"], utc=True)
    elif "asof" in df.columns:
        df["asof"] = pd.to_datetime(df["asof"], utc=True)
    else:
        raise ValueError("Neither 'date' nor 'asof' exists in df in add_neighbor_features")


    req = ['asof', 'pool_id', 'apy_7d', 'tvl_usd']
    for c in req:
        if c not in df.columns:
            print(f'Column {c} not found in panel_df.')
    # Normalize dates (daily) and sort
    df['asof'] = pd.to_datetime(df['asof'], utc=True).dt.normalize()
    df = df.sort_values(['asof', group_col, 'pool_id'])

    # -------- SAME-DAY (t) group stats (OK for t+1 forecasts) --------
    grp_t = df.groupby(['asof', group_col], dropna=False)

    group_tvl_sum_t   = grp_t['tvl_usd'].transform('sum')    .rename('group_tvl_sum_t_nbr')
    group_apy_mean_t  = grp_t['apy_7d'].transform('mean')    .rename('group_apy_mean_t_nbr')
    group_apy_median_t= grp_t['apy_7d'].transform('median')  .rename('group_apy_median_t_nbr')
    group_apy_std_t   = grp_t['apy_7d'].transform('std')     .rename('group_apy_std_t_nbr')

    df = pd.concat([df, group_tvl_sum_t, group_apy_mean_t, group_apy_median_t, group_apy_std_t], axis=1)

    # TVL share inside group at t
    denom = df['group_tvl_sum_t_nbr'].replace(0, np.nan)
    df['tvl_share_nbr'] = (df['tvl_usd'] / denom).fillna(0.0)

    # Normalized rank (0..1) of apy_7d within group at t (includes pool itself)
    def _rank_norm(x):
        r = x.rank(method='average', na_option='keep')
        return (r - 1) / max(len(r) - 1, 1)
    df['apy_rank_nbr'] = grp_t['apy_7d'].transform(_rank_norm)

    # Exclude-this-pool group mean at t
    group_count_t = grp_t['apy_7d'].transform('count').rename('group_count_t_nbr')
    group_sum_t   = grp_t['apy_7d'].transform('sum')  .rename('group_sum_t_nbr')
    df = pd.concat([df, group_count_t, group_sum_t], axis=1)

    excl_mean = (df['group_sum_t_nbr'] - df['apy_7d']) / (df['group_count_t_nbr'] - 1).replace(0, np.nan)
    df['grp_ex_mean_t_nbr'] = excl_mean.fillna(df['group_apy_mean_t_nbr'])

    # Build group-daily series for rolling calcs (dedup per (date, group))
    g_daily = (
        df[['asof', group_col, 'grp_ex_mean_t_nbr']]
        .drop_duplicates(['asof', group_col])
        .sort_values(['asof', group_col])
    )

    # 7d rolling of excl-mean (per group)
    g_daily['grp_ex_mean_7d_nbr'] = (
        g_daily.groupby(group_col)['grp_ex_mean_t_nbr']
               .transform(lambda s: s.rolling(7, min_periods=3).mean())
    )

    df = df.merge(g_daily[['asof', group_col, 'grp_ex_mean_7d_nbr']],
                  on=['asof', group_col], how='left')

    # -------- PAST-ONLY (t-1) variants (for ultra-conservative setups) --------
    # Collapse same-day stats to (date, group), then shift by 1 day per group.
    g_same = (
        df[['asof', group_col,
            'group_tvl_sum_t_nbr', 'group_apy_mean_t_nbr', 'group_apy_median_t_nbr',
            'group_apy_std_t_nbr', 'grp_ex_mean_t_nbr', 'grp_ex_mean_7d_nbr']]
        .drop_duplicates(['asof', group_col])
        .sort_values(['asof', group_col])
        .copy()
    )

    for col in ['group_tvl_sum_t_nbr', 'group_apy_mean_t_nbr', 'group_apy_median_t_nbr',
                'group_apy_std_t_nbr', 'grp_ex_mean_t_nbr', 'grp_ex_mean_7d_nbr']:
        g_same[col + '_lag1'] = (
            g_same.groupby(group_col)[col].shift(1)
        )

    df = df.merge(
        g_same[['asof', group_col] + [c + '_lag1' for c in
               ['group_tvl_sum_t_nbr','group_apy_mean_t_nbr','group_apy_median_t_nbr',
                'group_apy_std_t_nbr','grp_ex_mean_t_nbr','grp_ex_mean_7d_nbr']]],
        on=['asof', group_col],
        how='left'
    )

    # Final NA handling for neighbor block
    fill_cols = [
        'group_tvl_sum_t_nbr','group_apy_mean_t_nbr','group_apy_median_t_nbr','group_apy_std_t_nbr',
        'tvl_share_nbr','apy_rank_nbr','grp_ex_mean_t_nbr','grp_ex_mean_7d_nbr',
        'group_tvl_sum_t_nbr_lag1','group_apy_mean_t_nbr_lag1','group_apy_median_t_nbr_lag1',
        'group_apy_std_t_nbr_lag1','grp_ex_mean_t_nbr_lag1','grp_ex_mean_7d_nbr_lag1'
    ]
    for c in fill_cols:
        if c in df.columns:
            df[c] = df[c].fillna(0.0)

    return df

In [8]:
EXOG_BASE = ['eth_open', 
        'btc_open', 
        'gas_price_gwei', 
        'tvl_usd', 
        'apy_7d',
        '1-Month Treasury Yield', 
        '1-Year Treasury Bills Yield',
        '10-Year Treasury Minus 2-Year Treasury Spread',
        '10-Year Treasury Minus 3-Month Treasury Spread',
        '10-Year Treasury Yield', '2-Year Treasury Yield',
        '3-Month Treasury Yield', '30-Year Treasury Yield',
        '6-Month Treasury Yield',
        'Credit Suisse NASDAQ Gold FLOWS103 Price Index',
        'Federal Funds Effective Rate',
        'ICE BofA US High Yield Index Effective Yield',
        'M2 Not Seasonally Adjusted', 'M2 Seasonally Adjusted',
        'Nasdaq 100 Index', 'Nominal Broad U.S. Dollar Index',
        'Reverse Repo Yield (Overnight Reverse Repurchase Award Rate)',
        'S&P 500 Index (Daily Close)', 'SOFR 180-Day Average',
        'SOFR 30-Day Average', 'SOFR 90-Day Average', 'SOFR Index',
        'Secured Overnight Financing Rate']

LAG_SETS  = {
    'eth_open':        [7, 30],
    'btc_open':        [7, 30],
    'gas_price_gwei':  [7, 30],
    'tvl_usd':         [7, 30],
    'apy_7d':          [7, 30],
    '1-Month Treasury Yield': [7, 30],
    '1-Year Treasury Bills Yield': [7, 30],
    '10-Year Treasury Minus 2-Year Treasury Spread': [7, 30],
    '10-Year Treasury Minus 3-Month Treasury Spread': [7, 30],
    '10-Year Treasury Yield': [7, 30],
    '2-Year Treasury Yield': [7, 30],
    '3-Month Treasury Yield': [7, 30],
    '30-Year Treasury Yield': [7, 30],
    '6-Month Treasury Yield': [7, 30],
    'Credit Suisse NASDAQ Gold FLOWS103 Price Index': [7, 30],
    'Federal Funds Effective Rate': [7, 30],
    'ICE BofA US High Yield Index Effective Yield': [7, 30],
    'M2 Not Seasonally Adjusted': [7, 30],
    'M2 Seasonally Adjusted': [7, 30],
    'Nasdaq 100 Index': [7, 30],
    'Nominal Broad U.S. Dollar Index': [7, 30],
    'Reverse Repo Yield (Overnight Reverse Repurchase Award Rate)': [7, 30],
    'S&P 500 Index (Daily Close)': [7, 30],
    'SOFR 180-Day Average': [7, 30],
    'SOFR 30-Day Average': [7, 30],
    'SOFR 90-Day Average': [7, 30],
    'SOFR Index': [7, 30],
    'Secured Overnight Financing Rate': [7, 30]
}

def _stable_hash_0_1(s: str, mod: int = 1000) -> float:
    """Deterministic hash to [0,1) based on md5."""
    h = hashlib.md5(s.encode('utf-8')).hexdigest()
    val = int(h[:8], 16) % mod
    return val / float(mod)

def build_pool_feature_row(panel_df: pd.DataFrame,
                           pool_id: str,
                           asof: pd.Timestamp) -> dict:
    """
    Build leakage-safe feature row for (pool_id, asof) using only information
    available at end-of-day `asof`, for forecasting next-day actual_apy.

    - Uses preprocess_data(..., exogenous_cols=EXOG_BASE) which creates *_shifted
      versions of exogenous vars to avoid leakage.
    - Adds explicit lags from LAG_SETS.
    - Pulls neighbor features computed on `panel_df` for the same day (t) with *_nbr names.
    """

    # normalize tz
    asof = pd.Timestamp(asof)
    asof = asof.tz_localize('UTC') if asof.tz is None else asof.tz_convert('UTC')
    asof_day = asof.normalize()

    # history for this pool up to asof
    hist = (panel_df.loc[panel_df['pool_id'] == pool_id]
                    .sort_values('date')
                    .copy())

    if hist.empty:
        return {}

    # ensure datetime normalized
    hist['date'] = pd.to_datetime(hist['date'], utc=True).dt.normalize()
    hist = hist.set_index('date').sort_index()

    # require the asof day to be present (features at t to predict t+1)
    if asof_day not in hist.index:
        return {}

    # ---- base + exogenous (with _shifted from preprocess_data) ----
    feat = preprocess_data(hist.reset_index(), exogenous_cols=EXOG_BASE)

    # add explicit lags
    for col, lags in LAG_SETS.items():
        if col in feat.columns:
            feat = create_lagged_features(feat, col, lags)

    # keep only the row at `asof`
    if asof_day not in feat.index:
        return {}
    row = feat.loc[asof_day].to_dict()

    # ---- calendar (no leakage) ----
    dow = int(asof_day.dayofweek)
    doy = int(asof_day.dayofyear)
    row['dow_sin'] = np.sin(2*np.pi * dow / 7.0)
    row['dow_cos'] = np.cos(2*np.pi * dow / 7.0)
    row['doy_sin'] = np.sin(2*np.pi * doy / 365.25)
    row['doy_cos'] = np.cos(2*np.pi * doy / 365.25)

    # ---- stable pool id hash (0..1) ----
    row['pool_id_hash'] = _stable_hash_0_1(pool_id, mod=1000)

    return row

In [9]:
def classify_instability(row,low_thr,high_thr):
    if row["history_days"] < 7 or pd.isna(row["instability_3d"]):
        return "insufficient"

    val = row["instability_3d"]

    if val < low_thr:
        return "low"
    elif val < high_thr:
        return "medium"
    else:
        return "high"


def build_global_panel_dataset(asof_start: pd.Timestamp, asof_end: pd.Timestamp,
                               pool_ids: list,
                               hist_days:int=HIST_DAYS_PANEL) -> pd.DataFrame:
    """
    Build training rows with target = next-day actual_apy.
    Only include pools that have >=3 valid actual_apy values by `asof` (cold-start guard).
    """
    rows = []
    days = pd.date_range(asof_start, asof_end, freq='D')
    engine = get_db_connection()

    macro_daily = fetch_all_macro_daily(engine, 
                                        start_date=asof_start - pd.Timedelta(days=hist_days),
                                        end_date=asof_end)

    macro_daily = macro_daily.sort_values("date")
    macro_cols = [c for c in macro_daily.columns if c != "date"]

    # Example: 1d, 3d, 7d lags for all macro series
    for lag in [1, 3, 7]:
        lagged = macro_daily[macro_cols].shift(lag)
        lagged.columns = [f"{c}_lag{lag}" for c in macro_cols]
        macro_daily = pd.concat([macro_daily, lagged], axis=1)
        
    macro_daily = macro_daily.ffill()


                                
    for t in days:
        t = pd.Timestamp(t)
        t = t.tz_localize('UTC') if t.tz is None else t.tz_convert('UTC')

        panel = fetch_panel_history(t, pool_ids, days=hist_days)
        if panel.empty:
            continue

        panel = panel.merge(
            macro_daily,
            on="date",
            how="inner" 
        )

        # realized next day (target)
        with engine.connect() as conn:
            realized = pd.read_sql(
                text("""SELECT pool_id, date, actual_apy,actual_tvl
                        FROM pool_daily_metrics
                        WHERE date = :d"""),
                conn, params={"d": (t + pd.Timedelta(days=1)).normalize().to_pydatetime()}
            )
        if realized.empty:
            continue
        realized['date'] = pd.to_datetime(realized['date'], utc=True).dt.normalize()

        pools_today = panel.loc[panel['date'] == t.normalize(), 'pool_id'].unique().tolist()
        for pid in pools_today:

            n_hist = _count_actual_history(panel, pid, t.normalize())
            # inside the for pid loop
            hist_apys = panel.loc[
                (panel['pool_id'] == pid) &
                (panel['apy_7d'].notna()) &
                (panel['date'] <= t)
            ]['apy_7d']

            n_valid = len(hist_apys)
            if n_hist < 2:
                continue  # do not train on unstable early windows
            elif n_valid == 2:
                # simple average baseline
                baseline = hist_apys.mean()
                rows.append({
                    'pool_id': pid,
                    'asof': t.normalize(),
                    'target_apy_t1': np.nan,   # not used for training
                    'target_tvl_t1': np.nan,   # not used for training
                    'pred_global_apy': baseline,
                    'cold_start_flag': True
                })
                continue

            feat_row = build_pool_feature_row(panel, pid, t)
            if not feat_row:
                continue

            y_next_apy = realized.loc[realized['pool_id'] == pid, 'actual_apy']
            y_next_tvl = realized.loc[realized['pool_id'] == pid, 'actual_tvl'] 

            target_apy = float(y_next_apy.iloc[0]) if len(y_next_apy) > 0 and pd.notna(y_next_apy.iloc[0]) else np.nan
            target_tvl = float(y_next_tvl.iloc[0]) if len(y_next_tvl) > 0 and pd.notna(y_next_tvl.iloc[0]) else np.nan 

            if pd.isna(target_apy):
                continue

            feat_row.update({
                'pool_id': pid,
                'asof': t.normalize(),
                'target_apy_t1': target_apy,
                'target_tvl_t1': target_tvl,
                'pred_global_apy': np.nan,
                'cold_start_flag': False
            })
            rows.append(feat_row)

    panel = pd.DataFrame(rows)

    panel = panel.sort_values(['pool_id', 'asof']).reset_index(drop=True)
        
    # Add neighbor features based on instability groups
    panel = add_neighbor_features(panel, group_col='instability_group')

    return panel

In [10]:
def _get_feature_cols_for_training(df: pd.DataFrame) -> list:
    drop_like = {
        'target_apy_t1', 'target_tvl_t1',
        'pred_global_apy', 'cold_start_flag',
        'target_date', 'pool_group'  # if present
    }
    # numeric-only, excluding targets and helper columns
    cols = df.select_dtypes(include=[np.number]).columns.tolist()
    cols = [c for c in cols if c not in drop_like]
    return cols

def fit_global_tvl_model(train_df: pd.DataFrame, feature_cols: list) -> LGBMRegressor:
    """Train a simple LightGBM model for target_tvl_t1."""
    tvl_df = train_df.dropna(subset=['target_tvl_t1']).copy()
    X = tvl_df[feature_cols]
    y = tvl_df['target_tvl_t1'].astype(float)
    model = LGBMRegressor(
        objective='regression',
        n_estimators=800,
        num_leaves=64,
        learning_rate=0.03,
        subsample=0.9,
        colsample_bytree=0.9,
        random_state=123,
        force_col_wise=True,
        verbosity=-1
    )
    model.fit(X, y)
    return model



def _blocked_time_cv_mae(train_df: pd.DataFrame, feature_cols: list, params: dict, n_splits: int = 5) -> float:
    """Returns mean MAE over blocked time CV (like your OOF)."""
    df = train_df.sort_values('asof').reset_index(drop=True).copy()
    df = df.dropna(subset=['target_tvl_t1'])
    if df.empty:
        return float('inf')

    n = len(df)
    if n_splits < 2 or n < (n_splits + 1):
        # not enough data to validate properly
        n_splits = max(2, min(5, n // 10))  # degrade gracefully
        if n_splits < 2:
            return float('inf')

    fold_sizes = np.full(n_splits, n // n_splits, dtype=int)
    fold_sizes[: n % n_splits] += 1
    starts = np.cumsum(np.r_[0, fold_sizes[:-1]])
    ends = starts + fold_sizes

    maes = []
    for i in range(1, n_splits):
        tr = df.iloc[:ends[i-1]]
        vl = df.iloc[starts[i]:ends[i]]

        tr_ok = tr.dropna(subset=['target_tvl_t1'])
        vl_ok = vl.dropna(subset=['target_tvl_t1'])
        if tr_ok.empty or vl_ok.empty:
            continue

        X_tr = tr_ok[feature_cols]; y_tr = tr_ok['target_tvl_t1'].astype(float)
        X_vl = vl_ok[feature_cols]; y_vl = vl_ok['target_tvl_t1'].astype(float)

        model = LGBMRegressor(**params)
        # (Optional early stopping on time-ordered val)
        try:
            model.fit(X_tr, y_tr,
                      eval_set=[(X_vl, y_vl)],
                      eval_metric="l1",
                      callbacks=[]  # keep quiet and deterministic
                      )
        except Exception:
            model.fit(X_tr, y_tr)

        preds = model.predict(X_vl)
        maes.append(mean_absolute_error(y_vl, preds))

    return float(np.mean(maes)) if maes else float('inf')


def tune_tvl_params_with_optuna(train_df: pd.DataFrame,
                                feature_cols: list,
                                n_splits: int = 3,
                                n_trials: int = 30,
                                timeout: int | None = None,
                                seed: int = 123) -> dict:
    """Returns best LightGBM params found by Optuna (minimizes blocked-CV MAE)."""
    def objective(trial: optuna.Trial) -> float:
        params = dict(
            objective='regression',
            metric='l1',
            n_estimators=trial.suggest_int('n_estimators', 300, 1000),
            num_leaves=trial.suggest_int('num_leaves', 16, 256, log=True),
            max_depth=trial.suggest_int('max_depth', -1, 16),
            learning_rate=trial.suggest_float('learning_rate', 0.005, 0.08, log=True),
            subsample=trial.suggest_float('subsample', 0.6, 1.0),
            colsample_bytree=trial.suggest_float('colsample_bytree', 0.6, 1.0),
            min_data_in_leaf=trial.suggest_int('min_data_in_leaf', 5, 100, log=True),
            min_split_gain=trial.suggest_float('min_split_gain', 0.0, 0.5),
            reg_alpha=trial.suggest_float('reg_alpha', 0.0, 1.0),
            reg_lambda=trial.suggest_float('reg_lambda', 0.0, 5.0),
            force_col_wise=True,
            verbosity=-1,
            random_state=seed,
        )
        mae = _blocked_time_cv_mae(train_df, feature_cols, params, n_splits=n_splits)
        return mae

    study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=seed))
    study.optimize(objective, n_trials=n_trials, timeout=timeout, gc_after_trial=True)
    best_params = study.best_params

    # Fill fixed params not part of the search (for consistent fitting later)
    best_params.update(dict(
        objective='regression',
        metric='l1',
        force_col_wise=True,
        verbosity=-1,
        random_state=seed
    ))
    return best_params

def make_tvl_oof(train_df: pd.DataFrame,
                 feature_cols: list,
                 n_splits: int = 5,
                 use_optuna: bool = True,
                 n_trials: int = 15,
                 timeout: int | None = None,
                 seed: int = 123
                 ) -> tuple[pd.DataFrame, LGBMRegressor, float]:
    """
    Create leakage-free OOF predictions for target_tvl_t1 using blocked time CV.
    Optionally tune LGBM params with Optuna and use the best params for OOF + final model.

    Returns:
      - train_df with new column 'tvl_hat_t1_oof'
      - final tvl_model fitted on full data (with best params if tuned)
      - OOF MAE (avg over validation blocks)
    """
    df = train_df.sort_values('asof').reset_index(drop=True).copy()
    df['tvl_hat_t1_oof'] = np.nan

    # ---- choose params (either tuned or defaults) ----
    if use_optuna:
        best_params = tune_tvl_params_with_optuna(df, feature_cols,
                                                  n_splits=n_splits,
                                                  n_trials=n_trials,
                                                  timeout=timeout,
                                                  seed=seed)
    else:
        best_params = dict(
            objective='regression',
            n_estimators=800,
            num_leaves=64,
            learning_rate=0.03,
            subsample=0.9,
            colsample_bytree=0.9,
            min_data_in_leaf=5,
            min_split_gain=0.0,
            reg_alpha=0.0,
            reg_lambda=0.0,
            force_col_wise=True,
            verbosity=-1,
            random_state=seed
        )

    # ---- build blocked CV indices ----
    n = len(df)
    fold_sizes = np.full(n_splits, n // n_splits, dtype=int)
    fold_sizes[: n % n_splits] += 1
    starts = np.cumsum(np.r_[0, fold_sizes[:-1]])
    ends = starts + fold_sizes

    maes = []
    for i in range(1, n_splits):
        tr = df.iloc[:ends[i-1]]
        vl = df.iloc[starts[i]:ends[i]]

        tr_ok = tr.dropna(subset=['target_tvl_t1'])
        vl_ok = vl.dropna(subset=['target_tvl_t1'])
        if tr_ok.empty or vl_ok.empty:
            continue

        model = LGBMRegressor(**best_params)
        model.fit(tr_ok[feature_cols], tr_ok['target_tvl_t1'].astype(float))

        preds = model.predict(vl_ok[feature_cols])
        df.loc[vl_ok.index, 'tvl_hat_t1_oof'] = preds

        maes.append(mean_absolute_error(vl_ok['target_tvl_t1'].astype(float), preds))

    oof_mae = float(np.mean(maes)) if maes else np.nan

    # ---- fit final model on all available rows with target_tvl_t1 ----
    tvl_df = df.dropna(subset=['target_tvl_t1']).copy()
    final_model = LGBMRegressor(**best_params)
    final_model.fit(tvl_df[feature_cols], tvl_df['target_tvl_t1'].astype(float))

    # (nice) keep best params on the fitted model for later inspection
    final_model.best_params_ = best_params
    
    # Risk Group creation 
    df = df.sort_values(['pool_id', 'asof']).reset_index(drop=True)

   
    return df, final_model, oof_mae

In [11]:
def fetch_realized_for_date(target_date):
    """
    Fetch realized next-day outcomes (APY + TVL) for each pool for a given target_date (= t+1).
    This function uses the same 'pool_daily_metrics' source as before but explicitly 
    renames the columns for clarity and downstream use.
    """
    engine = get_db_connection()
    q = text("""
        SELECT
            pool_id,
            date AS target_date,
            actual_apy   AS target_apy_t1,
            actual_tvl      AS target_tvl_t1
        FROM pool_daily_metrics
        WHERE CAST(date AS DATE) = :tgt
    """)
    with engine.connect() as conn:
        df = pd.read_sql(q, conn, params={"tgt": pd.Timestamp(target_date).normalize().tz_localize(None)})
    
    # Normalize timestamp to UTC (consistent with rest of notebook)
    df["target_date"] = pd.to_datetime(df["target_date"]).dt.tz_localize("UTC").dt.normalize()
    
    # Select clean column order
    return df[["pool_id", "target_date", "target_apy_t1", "target_tvl_t1"]]

In [29]:
def fit_global_panel_model(panel_df, 
                           n_trials: int = 30,
                           val_frac: float = 0.3, 
                           random_state: int = 123):
    """
    Fit a RandomForest global model to predict target_apy_t1, with Optuna
    hyperparameter tuning on a time-based holdout.

    Returns:
      - best_model: fitted RandomForestRegressor on ALL data with best params
      - feat_cols: list of feature names
      - study: Optuna study (for inspection)
    """
    df = panel_df.copy()
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.dropna(subset=['target_apy_t1'])

    # Ensure sorted by time if 'asof' exists
    if 'asof' in df.columns:
        df = df.sort_values('asof')

    if "pool_id" in df.columns:
        le_pool_encoder = LabelEncoder()
        df["pool_id_code"] = le_pool_encoder.fit_transform(df["pool_id"].astype(str))

    # ---- Categorical → numeric code for instability_group ----
    if 'instability_group' in df.columns:
        instab_map = {
            "insufficient": -1,
            "low": 0,
            "medium": 1,
            "high": 2
        }
        df["instability_group_code"] = df["instability_group"].map(instab_map).astype("Int64")

    # ---- Feature selection ----
    drop_like = {
        'target_apy_t1', 'target_tvl_t1',  # targets
        'pred_global_apy', 'cold_start_flag',
        'target_date', 'asof', 'pool_id', 'hour'
    }

    feat_cols = [
        c for c in df.select_dtypes(include=[np.number]).columns
        if c not in drop_like
    ]

    X = df[feat_cols].fillna(0.0)
    y = df['target_apy_t1'].astype(float)

    # ---- Time-based train/valid split ----
    n = len(df)
    split_idx = int(n * (1.0 - val_frac))
    if split_idx <= 0 or split_idx >= n:
        split_idx = int(n * 0.8)

    X_train, X_valid = X.iloc[:split_idx], X.iloc[split_idx:]
    y_train, y_valid = y.iloc[:split_idx], y.iloc[split_idx:]

    # ---- Optuna objective ----
    def objective(trial: optuna.Trial) -> float:
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 200, 800),
            "max_depth": trial.suggest_int("max_depth", 1, 24),
            "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
            "max_features": trial.suggest_categorical(
                "max_features", ["sqrt", "log2", 0.3, 0.5, 0.7]
            ),
            "bootstrap": trial.suggest_categorical("bootstrap", [True, False]),
            "n_jobs": -1,
            "random_state": random_state,
        }

        model = RandomForestRegressor(**params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_valid)
        mae = mean_absolute_error(y_valid, y_pred)
        return mae  

    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=n_trials, show_progress_bar=False)
    

    best_params = study.best_params
    best_params.update({"n_jobs": -1, "random_state": random_state})

    # ---- Refit best model on ALL data ----
    best_model = RandomForestRegressor(**best_params)
    best_model.fit(X, y)

    print("Best RF params:", best_params)
    print("Best validation MAE:", study.best_value)

    return best_model, feat_cols, study,le_pool_encoder,df

In [28]:
def build_predict_panel(asof: pd.Timestamp,
                        pool_ids: list,
                        hist_days: int = HIST_DAYS_PANEL,
                        pool_id_encoder=None,
                        lag_pad_days: int = 7) -> pd.DataFrame:
    """
    Build the historical panel (per-pool time series + macro + lags) needed
    to create feature rows at a given `asof` for prediction.

    Notes:
    - This function does NOT compute neighbor features.
      Neighbor features must be computed on the final df_pred
      (1 row per pool), exactly as in training.
    """

    # --- Normalize asof ---
    t = pd.Timestamp(asof)
    t = t.tz_localize("UTC") if t.tz is None else t.tz_convert("UTC")

    # --- Fetch extended window (pad for rolling windows) ---
    effective_days = hist_days + lag_pad_days
    start = (t - pd.Timedelta(days=effective_days)).normalize()

    engine = get_db_connection()

    # 1) Raw per-pool history
    panel = fetch_panel_history(t, pool_ids, days=effective_days)
    if panel.empty:
        return panel

    panel["date"] = pd.to_datetime(panel["date"], utc=True).dt.normalize()

    # 2) Macro history + lags
    macro_daily = fetch_all_macro_daily(
        engine,
        start_date=start,
        end_date=t.normalize()
    )

    macro_daily = macro_daily.sort_values("date")
    macro_daily["date"] = pd.to_datetime(macro_daily["date"], utc=True).dt.normalize()

    macro_cols = [c for c in macro_daily.columns if c != "date"]

    for lag in [1, 3, 7, 30]:
        lagged = macro_daily[macro_cols].shift(lag)
        lagged.columns = [f"{c}_lag{lag}" for c in macro_cols]
        macro_daily = pd.concat([macro_daily, lagged], axis=1)

    macro_daily = macro_daily.ffill()

    # 3) Merge macros with pool time-series
    panel = panel.merge(macro_daily, on="date", how="inner")

    # 4) Encode pool_id using SAME encoder as training
    if pool_id_encoder is not None:
        known = panel["pool_id"].astype(str).isin(pool_id_encoder.classes_)
        panel.loc[known, "pool_id_code"] = pool_id_encoder.transform(
            panel.loc[known, "pool_id"].astype(str)
        )
        panel.loc[~known, "pool_id_code"] = -1  # unknown bucket
        panel["pool_id_code"] = panel["pool_id_code"].astype(int)

    # 5) Trim to last hist_days (drop the 7-day pad)
    cutoff_start = (t - pd.Timedelta(days=hist_days)).normalize()
    panel = panel.loc[panel["date"] >= cutoff_start].copy()

    # 6) Drop rows with missing actual_apy (needed for feature building)
    panel.dropna(subset=["actual_apy"], inplace=True)

    panel = add_neighbor_features(panel,"instability_group")

    # 7) Sort final panel
    panel = panel.sort_values(["pool_id", "date"]).reset_index(drop=True)

    return panel

In [35]:
def predict_global_head(
    asof: pd.Timestamp,
    pool_ids: list,
    model,                     # RandomForestRegressor (or similar)
    feat_cols: list,           # selected_features returned by fit_global_panel_model
    hist_days: int = HIST_DAYS_PANEL,
    tvl_model=None,            # optional TVL model (e.g. LightGBM)
    tvl_feat_cols: list | None = None,
    pool_id_encoder=None,      # <-- NEW: Encoder from training
) -> pd.DataFrame:
    """
    Predict next-day APY per pool for `asof + 1`.

    Supports:
    - RF global APY model
    - optional TVL stacking
    - instability classification
    - cold start handling (n_valid_apy = 2)
    """

    # ----------------------------------------
    # Normalize ASOF
    # ----------------------------------------
    asof = pd.Timestamp(asof)
    asof = asof.tz_localize("UTC") if asof.tz is None else asof.tz_convert("UTC")
    asof_norm = asof.normalize()
    target_date = asof_norm + pd.Timedelta(days=1)

    # ----------------------------------------
    # 1) Build historical panel
    #    (NO neighbor features here)
    # ----------------------------------------
    panel = build_predict_panel(
        asof=asof_norm,
        pool_ids=pool_ids,
        hist_days=hist_days,
        pool_id_encoder=pool_id_encoder,   # <-- FIX: apply encoding here
    )

    if panel.empty:
        return pd.DataFrame(
            columns=[
                "pool_id",
                "target_date",
                "pred_global_apy",
                "pred_global_apy_q10",
                "pred_global_tvl",
                "cold_start_flag",
                "instability_group",
                "pred_next_day_instability_group",
                "downside_risk",
            ]
        )

    # Map instability group to numeric like in training
    instab_map = {"insufficient": -1, "low": 0, "medium": 1, "high": 2}

    def _map_change_to_group(delta: float) -> str | float:
        if pd.isna(delta):
            return np.nan
        ad = abs(delta)
        if ad < 1:
            return "low"
        elif ad < 3:
            return "medium"
        else:
            return "high"

    # ----------------------------------------
    # Pools that exist today (same logic as training)
    # ----------------------------------------
    pools_today = panel.loc[panel["date"] == asof_norm, "pool_id"].unique().tolist()
    if not pools_today:
        return pd.DataFrame()

    rows = []

    # ----------------------------------------
    # 2) Loop through pools today
    # ----------------------------------------

    for pid in pools_today:

        # ---------- History counts ----------
        n_hist = _count_actual_history(panel, pid, asof_norm)

        hist_apys = panel.loc[
            (panel["pool_id"] == pid)
            & (panel["apy_7d"].notna())
            & (panel["date"] <= asof_norm)
        ]["apy_7d"]

        hist_tvl = panel.loc[
            (panel["pool_id"] == pid)
            & (panel["tvl_usd"].notna())
            & (panel["date"] <= asof_norm)
        ]["tvl_usd"]

        n_valid_apy = len(hist_apys)
        n_valid_tvl = len(hist_tvl)

        # previous-day APY (for instability)
        prev_hist = panel.loc[
            (panel["pool_id"] == pid)
            & panel["actual_apy"].notna()
            & (panel["date"] <= asof_norm)
        ].sort_values("date")

        actual_apy_prev_day = float(prev_hist["actual_apy"].iloc[-1]) if len(prev_hist) > 0 else np.nan

        # =============================================================
        # COLD START BRANCH (same as training)
        # =============================================================
        if n_hist < 2:
            continue

        if n_valid_apy == 2:
            baseline_apy = float(hist_apys.tail(2).mean())
            baseline_tvl = float(hist_tvl.tail(2).mean()) if n_valid_tvl >= 2 else np.nan

            # classify APY change
            delta = (baseline_apy - actual_apy_prev_day) if not pd.isna(actual_apy_prev_day) else np.nan
            pred_group = _map_change_to_group(delta)

            rows.append({
                "pool_id": pid,
                "target_date": target_date,
                "pred_global_apy": baseline_apy,
                "pred_global_apy_q10": baseline_apy,
                "pred_global_apy_risk_adj": baseline_apy,
                "pred_global_tvl": baseline_tvl,
                "cold_start_flag": True,
                "instability_group": "insufficient",
                "pred_next_day_instability_group": pred_group,
                "downside_risk": 0.0,
            })
            continue

        # =============================================================
        # MAIN MODEL BRANCH
        # =============================================================

        # --- Build feature row from history ---
        feat_row = build_pool_feature_row(panel, pid, asof_norm)
        if not feat_row:
            continue

        fr = pd.DataFrame([feat_row])

        # --- Encode pool_id if needed ---
        if pool_id_encoder is not None:
            if "pool_id_code" not in fr.columns:
                if str(pid) in pool_id_encoder.classes_:
                    fr["pool_id_code"] = pool_id_encoder.transform([str(pid)])[0]
                else:
                    fr["pool_id_code"] = -1

        # --- Instability group code ---
        if "instability_group" in fr.columns:
            fr["instability_group_code"] = fr["instability_group"].map(instab_map).astype("Int64")


        # ========== Optional TVL stacking ==========
        tvl_hat = np.nan
        if (tvl_model is not None) and (tvl_feat_cols is not None):
            for c in tvl_feat_cols:
                if c not in fr.columns:
                    fr[c] = 0.0

            X_tvl = fr[tvl_feat_cols].fillna(0.0)
            tvl_hat = float(tvl_model.predict(X_tvl)[0])
            fr["tvl_hat_t1_oof"] = tvl_hat


        if pool_id_encoder is not None:
            if "pool_id_code" not in fr.columns:
                if str(pid) in pool_id_encoder.classes_:
                    fr["pool_id_code"] = pool_id_encoder.transform([str(pid)])[0]
                else:
                    fr["pool_id_code"] = -1 

        # ========== Align to training features ==========
        for c in feat_cols:
            if c not in fr.columns:
                print(c)
                fr[c] = 0.0

        fr_pred = fr[feat_cols].fillna(0.0)

        # --- APY prediction ---
        yhat = float(model.predict(fr_pred)[0])

        # --- q10 quantile from RF trees ---
        if hasattr(model, "estimators_"):
            X_np = fr_pred.to_numpy()
            preds_all = np.stack([tree.predict(X_np) for tree in model.estimators_], axis=1)
            q10 = float(np.quantile(preds_all, 0.10, axis=1)[0])
        else:
            q10 = yhat

        downside = (yhat - q10) / (yhat + 1e-9)

        # predicted next-day instability group
        delta = yhat - actual_apy_prev_day if not pd.isna(actual_apy_prev_day) else np.nan
        pred_group = _map_change_to_group(delta)

        # instability group (current)
        if "instability_group_code" in fr.columns:
            inv_map = {v: k for k, v in instab_map.items()}
            instab_group = inv_map.get(int(fr["instability_group_code"].iloc[0]), "insufficient")
        else:
            instab_group = "insufficient"

        gamma = 0.5  # tune via backtest

        apy_risk_adj = (1 - gamma) * yhat + gamma * q10

        rows.append({
            "pool_id": pid,
            "target_date": target_date,
            "pred_global_apy": yhat,
            "pred_global_apy_q10": q10,
            "pred_global_apy_risk_adj": apy_risk_adj,
            "pred_global_tvl": tvl_hat,
            "cold_start_flag": False,
            "instability_group": instab_group,
            "pred_next_day_instability_group": pred_group,
            "downside_risk": downside,
        })
    missing_cols = [c for c in feat_cols if c not in fr.columns]
    if missing_cols:
        print(f"[WARN] Missing {len(missing_cols)} features for pool {pid} on {asof_norm}:")
        print(missing_cols)

    return pd.DataFrame(rows),panel

# Global LGBM + RF entire flow for APY and TVL prediction

In [ ]:
TRAIN_WINDOW = 60
END_DATE   = pd.Timestamp.utcnow().normalize() - pd.Timedelta(days=5)
START_DATE = END_DATE - pd.Timedelta(days=TRAIN_WINDOW) 


In [25]:
# 1) Get pool ids 
pool_ids = get_filtered_pool_ids_readonly(limit=200)  # your helper

# 2) Build panel training for tvl 
panel_train_tvl = build_global_panel_dataset(
    asof_start=START_DATE,
    asof_end=END_DATE,
    pool_ids=pool_ids,
    hist_days=HIST_DAYS_PANEL
)

# 3) Train tvl model
feat_cols = _get_feature_cols_for_training(panel_train_tvl)
panel_with_oof, tvl_model, oof_mae = make_tvl_oof(panel_train_tvl, feat_cols, n_splits=5)


[I 2025-12-03 18:09:00,454] A new study created in memory with name: no-name-e22fe253-2dc9-41c0-b1e2-a1500ffe1b26
[I 2025-12-03 18:09:04,527] Trial 0 finished with value: 9229569.338932255 and parameters: {'n_estimators': 788, 'num_leaves': 35, 'max_depth': 3, 'learning_rate': 0.023057867250559985, 'subsample': 0.8877875879142252, 'colsample_bytree': 0.7692425840497844, 'min_data_in_leaf': 95, 'min_split_gain': 0.34241486929243165, 'reg_alpha': 0.48093190148436094, 'reg_lambda': 1.9605875909707526}. Best is trial 0 with value: 9229569.338932255.
[I 2025-12-03 18:09:10,326] Trial 1 finished with value: 6353629.477385355 and parameters: {'n_estimators': 540, 'num_leaves': 120, 'max_depth': 6, 'learning_rate': 0.0058996921733422926, 'subsample': 0.7592177021321725, 'colsample_bytree': 0.8951981622928142, 'min_data_in_leaf': 8, 'min_split_gain': 0.08772587807374627, 'reg_alpha': 0.5315513738418384, 'reg_lambda': 2.6591379354843303}. Best is trial 1 with value: 6353629.477385355.
[I 2025-12

In [27]:
print("Panel rows:", len(panel_with_oof))

# 5) Fit the global model
#glob_model, glob_feats,study = fit_global_panel_model(panel_with_oof)
best_model, selected_features, study , le_encoder,df = fit_global_panel_model(panel_with_oof, n_trials=2)
print("Global model trained. #features:", len(selected_features))


[I 2025-12-03 18:17:12,100] A new study created in memory with name: no-name-3380ca5f-0e52-40e2-ab78-b135d6a0b52d


Panel rows: 7718


[I 2025-12-03 18:17:12,778] Trial 0 finished with value: 35.08426559348328 and parameters: {'n_estimators': 620, 'max_depth': 7, 'min_samples_split': 7, 'min_samples_leaf': 2, 'max_features': 'log2', 'bootstrap': False}. Best is trial 0 with value: 35.08426559348328.
[I 2025-12-03 18:17:30,200] Trial 1 finished with value: 22.372851361228797 and parameters: {'n_estimators': 425, 'max_depth': 16, 'min_samples_split': 13, 'min_samples_leaf': 9, 'max_features': 0.7, 'bootstrap': False}. Best is trial 1 with value: 22.372851361228797.


Best RF params: {'n_estimators': 425, 'max_depth': 16, 'min_samples_split': 13, 'min_samples_leaf': 9, 'max_features': 0.7, 'bootstrap': False, 'n_jobs': -1, 'random_state': 123}
Best validation MAE: 22.372851361228797
Global model trained. #features: 191


In [36]:
# 6) Make predictions
pred_global,panel = predict_global_head(
    asof=END_DATE,
    pool_ids=pool_ids,
    model=best_model,
    feat_cols=selected_features,
    hist_days=HIST_DAYS_PANEL,
    tvl_model=tvl_model,         
    tvl_feat_cols=feat_cols,
    pool_id_encoder = le_encoder        
)

In [37]:
pred_global

,pool_id,target_date,pred_global_apy,pred_global_apy_q10,pred_global_apy_risk_adj,pred_global_tvl,cold_start_flag,instability_group,pred_next_day_instability_group,downside_risk
0,00d50ef9-5980-447f-b1ca-ec651d34d111,2025-11-29 00:00:00+00:00,3.913954,3.763324,3.838639,8.520163e+06,False,low,low,0.038485
1,027b72c2-bf96-451b-ac5f-afe6859b0828,2025-11-29 00:00:00+00:00,4.672782,3.858793,4.265787,1.148835e+07,False,medium,low,0.174198
2,043a8330-bc29-4164-aa1c-28de7bf87755,2025-11-29 00:00:00+00:00,3.809322,3.449286,3.629304,8.702229e+05,False,medium,low,0.094515
3,050c0de3-e50b-4c2c-95ec-ae69effd77f1,2025-11-29 00:00:00+00:00,3.893212,3.733556,3.813384,5.755676e+05,False,low,low,0.041009
4,0fabba51-a1c1-4ca0-93a7-9735ca6e1c28,2025-11-29 00:00:00+00:00,3.849369,3.465744,3.657556,9.637158e+05,False,low,low,0.099659
...,...,...,...,...,...,...,...,...,...,...
97,f69e8b5b-c758-4361-b4aa-05d7e68ba4c4,2025-11-29 00:00:00+00:00,7.498854,6.827130,7.162992,1.507822e+06,False,low,low,0.089577
98,f88f32f0-fa63-4dff-b99a-6bf6bec3a00d,2025-11-29 00:00:00+00:00,3.917023,2.485591,3.201307,1.072281e+05,False,medium,low,0.365439
99,f918753b-4cf3-4eec-97aa-ab6b5409b5f9,2025-11-29 00:00:00+00:00,0.248582,0.000000,0.124291,1.132112e+05,False,insufficient,low,1.000000
100,feac7518-a0a7-4378-ba79-b1b9f4293525,2025-11-29 00:00:00+00:00,0.640227,0.000000,0.320113,3.920650e+05,False,insufficient,low,1.000000


# 📘 Prediction Output — Column Definitions

### `pool_id`
Identifier of the liquidity pool for which the prediction is generated.

### `target_date`
The date **for which the APY/TLV prediction applies** (always `asof + 1 day`).

---

## 🔵 APY Forecast Outputs

### `pred_global_apy`
Main predicted **APY** for the next day using the global ML model.  
Represents the model’s best estimate of tomorrow’s yield.

### `pred_global_apy_q10`
Conservative **10th-percentile APY forecast** (lower-bound scenario).  
Useful for downside or risk-averse decisions.

### `pred_global_apy_risk_adj`
A higher γ produces a more conservative estimate, useful for risk-aware 
pool selection, downside protection, and capital preservation strategies.

### `pred_next_day_instability_group`
Forecasted **volatility category** for tomorrow based on the predicted APY move:

| Category | Threshold | Meaning |
|---------|-----------|---------|
| **low** | abs(change) < 1 | small movement expected |
| **medium** | 1 ≤ abs(change) < 3 | moderate movement expected |
| **high** | abs(change) ≥ 3 | large movement expected |

⚠️ **NaN on the first prediction day for each pool** — no previous actual APY exists to compute a change.

### `downside_risk`
Relative downside deviation between main forecast and q10 scenario:

```
downside_risk = (pred_global_apy - pred_global_apy_q10) / pred_global_apy
```

Higher values indicate greater risk of a negative APY surprise.

---

## 🟢 TVL Forecast Outputs

### `pred_global_tvl`
Predicted total value locked (TVL) for the next day.

---

## ⚙️ Model / Status Flags

### `cold_start_flag`
Indicates pools with insufficient history (<3 valid APYs).  
If `True`, the prediction uses a **simple average baseline** instead of the learned model.

### `instability_group`
The **current stability regime** of the pool based on past 3‑day APY volatility:  
`low`, `medium`, `high`, or `insufficient`.


# TO BE IGNORED (Backtesting purposes) 

In [39]:
# realized outcomes for the predicted target_date
realized = fetch_realized_for_date(pred_global['target_date'].iloc[0])

# Merge predictions with realized
cmp = pred_global.merge(
    realized[['pool_id', 'target_apy_t1', 'target_tvl_t1']],
    on='pool_id', how='left'
)

# --- APY errors ---
cmp['ae_apy']   = (cmp['pred_global_apy'] - cmp['target_apy_t1']).abs()
cmp['mape_apy'] = (cmp['ae_apy'] / (cmp['target_apy_t1'].abs() + 1e-9)).clip(upper=10.0)

# --- TVL errors (only for rows where we have a TVL prediction) ---
cmp['ae_tvl']   = (cmp['pred_global_tvl'] - cmp['target_tvl_t1']).abs()
cmp['mape_tvl'] = (cmp['ae_tvl'] / (cmp['target_tvl_t1'].abs() + 1e-9)).clip(upper=10.0)

# Overall metrics
overall = pd.DataFrame({
    'n_pools': [len(cmp)],
    'APY_MAE': [cmp['ae_apy'].mean()],
    'APY_MAPE':[cmp['mape_apy'].mean()],
    'TVL_MAE': [cmp['ae_tvl'].mean(skipna=True)],
    'TVL_MAPE':[cmp['mape_tvl'].mean(skipna=True)],
    'TVL_cov':[cmp['pred_global_tvl'].notna().mean()]  # coverage of TVL preds (cold-start rows may be NaN)
})
display(overall)

# Per-group diagnostics (optional)
by_group = (cmp
    .groupby('pred_next_day_instability_group', dropna=False)
    .agg(
        n=('pool_id','count'),
        APY_MAE=('ae_apy','mean'),
        APY_MAPE=('mape_apy','mean'),
        TVL_MAE=('ae_tvl','mean'),
        TVL_MAPE=('mape_tvl','mean'),
        AE_TVL_MAX=('ae_tvl','max'),
        AE_APY_MAX=('ae_apy','max'),
        MEAN_APY=('pred_global_apy','mean'),
        MAX_APY=('target_apy_t1','max')
        

    )
    .sort_values('APY_MAE', ascending=False)
)
display(by_group.head(20))

# Top misses (APY + TVL)
display(
    cmp.assign(ae_tvl_filled=cmp['ae_tvl'].fillna(-1))  # keep rows with NaN TVL at the bottom
       .sort_values(['ae_apy','ae_tvl_filled'], ascending=False)
       .head(20)
)

,n_pools,APY_MAE,APY_MAPE,TVL_MAE,TVL_MAPE,TVL_cov
0,102,0.899185,1.279676,990479.442751,1.005684,1.0


,n,APY_MAE,APY_MAPE,TVL_MAE,TVL_MAPE,AE_TVL_MAX,AE_APY_MAX,MEAN_APY,MAX_APY
pred_next_day_instability_group,,,,,,,,,
high,7,5.151876,0.731355,2.606174e+05,1.919866,1.506912e+06,17.698785,29.457181,73.6200
medium,12,1.666826,1.010730,1.026087e+05,0.882816,4.507702e+05,7.559579,6.113001,14.7606
low,83,0.423813,1.365842,1.185090e+06,0.944884,2.519990e+07,3.472455,4.252154,11.7009


,pool_id,target_date,pred_global_apy,pred_global_apy_q10,pred_global_apy_risk_adj,pred_global_tvl,cold_start_flag,instability_group,pred_next_day_instability_group,downside_risk,target_apy_t1,target_tvl_t1,ae_apy,mape_apy,ae_tvl,mape_tvl,ae_tvl_filled
34,547216d2-86af-466b-a955-128e0fa0563d,2025-11-29 00:00:00+00:00,55.921215,39.940700,47.930958,1.144654e+05,False,high,high,0.285768,73.6200,39005.0,17.698785,0.240407,7.546038e+04,1.934634,7.546038e+04
62,8f331633-47cd-4ee6-a231-e0dba8978f9a,2025-11-29 00:00:00+00:00,7.201021,4.219198,5.710109,4.261672e+05,False,high,medium,0.414083,14.7606,242316.0,7.559579,0.512146,1.838512e+05,0.758725,1.838512e+05
14,2b181181-f200-44d0-81bf-31f3575fea4c,2025-11-29 00:00:00+00:00,12.533548,6.055920,9.294734,1.723845e+06,False,high,high,0.516823,7.2601,216933.0,5.273448,0.726360,1.506912e+06,6.946440,1.506912e+06
101,ffa56b1b-0366-4216-939e-7ac8181ed56c,2025-11-29 00:00:00+00:00,6.581451,3.614114,5.097782,1.072281e+05,False,medium,high,0.450864,1.4546,22873.0,5.126851,3.524578,8.435510e+04,3.687977,8.435510e+04
43,6599e2eb-6e11-4fd0-bbfd-1c08289882fe,2025-11-29 00:00:00+00:00,4.141135,1.899690,3.020412,1.998929e+05,False,high,high,0.541264,9.2407,168662.0,5.099565,0.551859,3.123091e+04,0.185169,3.123091e+04
8,17d7e243-a474-46c1-99f9-b3947bf0081c,2025-11-29 00:00:00+00:00,9.925555,8.373608,9.149581,1.072281e+05,False,low,low,0.156359,6.4531,130615.0,3.472455,0.538106,2.338690e+04,0.179052,2.338690e+04
44,659de03b-a9fd-41ee-8a1c-7c7de1c6782a,2025-11-29 00:00:00+00:00,5.242871,3.746143,4.494507,9.726321e+05,False,medium,low,0.285479,2.1055,1072718.0,3.137371,1.490084,1.000859e+05,0.093301,1.000859e+05
68,9595a47e-4954-424e-8a6d-044ceb3ccae4,2025-11-29 00:00:00+00:00,10.567735,8.546126,9.556930,8.702229e+05,False,medium,medium,0.191300,8.2508,860760.0,2.316935,0.280813,9.462877e+03,0.010994,9.462877e+03
80,b5ca1343-0d19-4a1c-8d0f-a2d2ffbe2cf8,2025-11-29 00:00:00+00:00,60.390897,36.297827,48.344362,6.908786e+05,False,high,high,0.398952,62.6216,633516.0,2.230703,0.035622,5.736263e+04,0.090546,5.736263e+04
28,422cbad0-7a9a-45c2-abd0-27b395d4c461,2025-11-29 00:00:00+00:00,4.457183,2.231370,3.344276,2.889622e+06,False,high,low,0.499377,2.5082,2041240.0,1.948983,0.777045,8.483819e+05,0.415621,8.483819e+05


# BackTest

In [23]:
# --- Rolling daily backtest with refit per day (train -> predict) ---
# Assumes these helpers already exist in your notebook:
# - build_global_panel_dataset
# - _get_numeric_feature_cols (or use the inline variant below)
# - make_tvl_oof, fit_global_tvl_model, fit_global_panel_model
# - predict_global_lgbm_head
# - fetch_realized_for_date

# Safe tqdm import
try:
    from tqdm.auto import tqdm
except Exception:
    def tqdm(x, **kwargs): return x

def _get_numeric_feature_cols(df, extra_drop=None):
    drop_cols = {
        'pool_id','asof','target_date',
        'target_apy_t1','target_tvl_t1',
        'pred_global_apy','cold_start_flag',
    }
    if extra_drop: drop_cols |= set(extra_drop)
    return [c for c in df.select_dtypes(include=[np.number]).columns if c not in drop_cols]

# --- config ---
TRAIN_WINDOW_DAYS = 60         # lookback window to build training set each day
BACKTEST_DAYS     = 60       # how many days to evaluate
USE_TVL_STACKING  = True       # turn stacking on/off

bt_end   = END_DATE.normalize()
bt_start = bt_end - pd.Timedelta(days=BACKTEST_DAYS - 1)
bt_range = pd.date_range(bt_start, bt_end, freq="D")

daily_rows = []
pool_rows  = []

print(f"Rolling backtest with daily refit | train_window={TRAIN_WINDOW_DAYS}d | period {bt_start.date()} → {bt_end.date()}")

for asof in tqdm(bt_range, desc="Backtesting (daily refit)"):
    # ---- 1) Build training window: [asof - TRAIN_WINDOW_DAYS .. asof - 1]
    train_start = (asof - pd.Timedelta(days=TRAIN_WINDOW_DAYS))
    train_end   = (asof - pd.Timedelta(days=1))
    if train_start >= train_end:
        continue

    panel_train = build_global_panel_dataset(
        asof_start=train_start,
        asof_end=train_end,
        pool_ids=pool_ids,
        hist_days=HIST_DAYS_PANEL
    )
    if panel_train.empty: 
        continue

    # ---- 2) Train TVL model (+OOF for stacking)
    TVL_FEATURE_COLS = _get_numeric_feature_cols(panel_train, extra_drop={'tvl_hat_t1_oof'})
    tvl_model = None
    train_for_apy = panel_train.copy()

    if USE_TVL_STACKING:
        train_for_apy, tvl_model, tvl_oof_mae = make_tvl_oof(panel_train, TVL_FEATURE_COLS, n_splits=5)
    else:
        tvl_model = fit_global_tvl_model(panel_train, TVL_FEATURE_COLS)
        tvl_oof_mae = np.nan

    # ---- 3) Train APY model (uses tvl_hat_t1_oof if present)
    apy_model, APY_FEATURE_COLS,study = fit_global_panel_model(train_for_apy)

    # ---- 4) Predict for asof (target is asof+1)
    preds = predict_global_lgbm_head(
        asof=asof,
        pool_ids=pool_ids,
        model=apy_model,
        feat_cols=APY_FEATURE_COLS,
        hist_days=HIST_DAYS_PANEL,
        tvl_model=tvl_model if USE_TVL_STACKING else tvl_model,        # ok in both cases
        tvl_feat_cols=TVL_FEATURE_COLS
    )
    if preds.empty:
        continue

    # ---- 5) Merge realized next-day outcomes
    realized = fetch_realized_for_date(preds["target_date"].iloc[0])
    cmp = preds.merge(
        realized[["pool_id","target_apy_t1","target_tvl_t1"]],
        on="pool_id", how="left"
    )

    # ---- 6) Errors
    for col in ["pred_global_apy","target_apy_t1","pred_global_tvl","target_tvl_t1"]:
        cmp[col] = pd.to_numeric(cmp[col], errors="coerce")

    
    cmp["ae_apy"]   = (cmp["pred_global_apy"] - cmp["target_apy_t1"]).abs()
    cmp["mape_apy"] = (cmp["ae_apy"] / (cmp["target_apy_t1"].abs() + 1e-9)).clip(upper=10.0)

    cmp["ae_tvl"] = (cmp["pred_global_tvl"] - cmp["target_tvl_t1"]).abs()

    def _safe_mape(ae, denom, eps=1e-9, cap=10.0):
        d = np.abs(pd.to_numeric(denom, errors="coerce"))
        out = ae / (d + eps)
        return out.clip(upper=cap)

    cmp["mape_apy"] = _safe_mape(cmp["ae_apy"], cmp["target_apy_t1"])
    cmp["mape_tvl"] = _safe_mape(cmp["ae_tvl"], cmp["target_tvl_t1"])

    # daily aggregate
    daily_rows.append({
        "date": asof.normalize(),
        "n_pools": len(cmp),
        "APY_MAE": cmp["ae_apy"].mean(),
        "APY_MAPE": cmp["mape_apy"].mean(),
        "TVL_MAE": cmp["ae_tvl"].mean(skipna=True),
        "TVL_MAPE": cmp["mape_tvl"].mean(skipna=True),
        "max_AE_APY": cmp["ae_apy"].max(),
        "max_AE_TVL": cmp["ae_tvl"].max(),
        "tvl_oof_mae_train": tvl_oof_mae,
        "used_stacking": USE_TVL_STACKING,
        "train_rows": len(panel_train)
    })

    # keep per-pool (and daily max AE per pool)
    cmp["asof"]   = asof.normalize()
    cmp["max_AE"] = cmp[["ae_apy","ae_tvl"]].max(axis=1)
    pool_rows.append(cmp[[
        "asof","pool_id",
        "pred_global_apy","pred_global_apy_q10","target_apy_t1","ae_apy","mape_apy",
        "pred_global_tvl","target_tvl_t1","ae_tvl","mape_tvl",
        "max_AE","cold_start_flag","instability_group","downside_risk"
    ]])

# ---- 7) Combine results
bt_daily_refit = pd.DataFrame(daily_rows).sort_values("date")
bt_pool_refit  = pd.concat(pool_rows, ignore_index=True) if pool_rows else pd.DataFrame()

if not bt_pool_refit.empty:
    bt_instab_daily = (
        bt_pool_refit
        .groupby(["asof", "instability_group"], as_index=False)
        .agg(
            n_pools     = ("pool_id", "nunique"),
            APY_MAE     = ("ae_apy", "mean"),
            APY_MAPE    = ("mape_apy", "mean"),
            TVL_MAE     = ("ae_tvl", "mean"),
            TVL_MAPE    = ("mape_tvl", "mean"),
            MAX_AE_APY  = ("ae_apy", "max"),
            MAX_AE_TVL  = ("ae_tvl", "max"),
        )
        .sort_values(["asof", "instability_group"])
    )
else:
    bt_instab_daily = pd.DataFrame()

overall_refit = pd.DataFrame([{
    "n_days": len(bt_daily_refit),
    "n_obs": len(bt_pool_refit),
    "APY_MAE": bt_pool_refit["ae_apy"].mean(),
    "APY_MAPE": bt_pool_refit["mape_apy"].mean(),
    "TVL_MAE": bt_pool_refit["ae_tvl"].mean(),
    "TVL_MAPE": bt_pool_refit["mape_tvl"].mean(),
    "MAX_AE_APY": bt_pool_refit["ae_apy"].max(),
    "MAX_AE_TVL": bt_pool_refit["ae_tvl"].max(),
}])

by_pool_refit = (bt_pool_refit
    .groupby("pool_id", as_index=False)
    .agg(
        n_days=("asof","nunique"),
        APY_MAE=("ae_apy","mean"),
        APY_MAPE=("mape_apy","mean"),
        TVL_MAE=("ae_tvl","mean"),
        TVL_MAPE=("mape_tvl","mean"),
        MAX_AE=("max_AE","max"),
        group_col=('instability_group',"first")
    )
    .sort_values("APY_MAE", ascending=False)
)

print("=== Overall (daily refit) ===")
display(overall_refit)
print("=== Daily (head) ===")
display(bt_daily_refit.head())
print("=== Worst pools (head) ===")
display(by_pool_refit.head())

Rolling backtest with daily refit | train_window=60d | period 2025-09-10 → 2025-11-08


Backtesting (daily refit):   0%|          | 0/60 [00:00<?, ?it/s][I 2025-11-16 01:54:38,069] A new study created in memory with name: no-name-ba68b09c-6e0b-4e81-a60d-46de5b5a1410
[I 2025-11-16 01:54:39,046] Trial 0 finished with value: 630651.6938041291 and parameters: {'n_estimators': 788, 'num_leaves': 35, 'max_depth': 3, 'learning_rate': 0.023057867250559985, 'subsample': 0.8877875879142252, 'colsample_bytree': 0.7692425840497844, 'min_data_in_leaf': 95, 'min_split_gain': 0.34241486929243165, 'reg_alpha': 0.48093190148436094, 'reg_lambda': 1.9605875909707526}. Best is trial 0 with value: 630651.6938041291.
[I 2025-11-16 01:54:41,909] Trial 1 finished with value: 349824.7369709284 and parameters: {'n_estimators': 540, 'num_leaves': 120, 'max_depth': 6, 'learning_rate': 0.0058996921733422926, 'subsample': 0.7592177021321725, 'colsample_bytree': 0.8951981622928142, 'min_data_in_leaf': 8, 'min_split_gain': 0.08772587807374627, 'reg_alpha': 0.5315513738418384, 'reg_lambda': 2.65913793548

Best RF params: {'n_estimators': 308, 'max_depth': 17, 'min_samples_split': 7, 'min_samples_leaf': 4, 'max_features': 0.7, 'bootstrap': True, 'n_jobs': -1, 'random_state': 123}
Best validation MAE: 0.7852812667126652


Backtesting (daily refit):   2%|▏         | 1/60 [01:49<1:47:56, 109.77s/it][I 2025-11-16 01:56:27,871] A new study created in memory with name: no-name-c362058a-e0fd-4a2e-b521-5e4594fc98ee
[I 2025-11-16 01:56:28,865] Trial 0 finished with value: 652940.82872226 and parameters: {'n_estimators': 788, 'num_leaves': 35, 'max_depth': 3, 'learning_rate': 0.023057867250559985, 'subsample': 0.8877875879142252, 'colsample_bytree': 0.7692425840497844, 'min_data_in_leaf': 95, 'min_split_gain': 0.34241486929243165, 'reg_alpha': 0.48093190148436094, 'reg_lambda': 1.9605875909707526}. Best is trial 0 with value: 652940.82872226.
[I 2025-11-16 01:56:32,195] Trial 1 finished with value: 366578.2365160304 and parameters: {'n_estimators': 540, 'num_leaves': 120, 'max_depth': 6, 'learning_rate': 0.0058996921733422926, 'subsample': 0.7592177021321725, 'colsample_bytree': 0.8951981622928142, 'min_data_in_leaf': 8, 'min_split_gain': 0.08772587807374627, 'reg_alpha': 0.5315513738418384, 'reg_lambda': 2.6591

Best RF params: {'n_estimators': 713, 'max_depth': 19, 'min_samples_split': 18, 'min_samples_leaf': 5, 'max_features': 0.7, 'bootstrap': True, 'n_jobs': -1, 'random_state': 123}
Best validation MAE: 0.8173415750719535


...

,n_days,n_obs,APY_MAE,APY_MAPE,TVL_MAE,TVL_MAPE,MAX_AE_APY,MAX_AE_TVL
0,60,1974,2.028685,0.118114,1.858845e+06,0.552438,170.000498,7.331656e+07


=== Daily (head) ===


,date,n_pools,APY_MAE,APY_MAPE,TVL_MAE,TVL_MAPE,max_AE_APY,max_AE_TVL,tvl_oof_mae_train,used_stacking,train_rows
0,2025-09-10 00:00:00+00:00,26,1.641451,0.153066,290351.009111,0.133067,7.035720,1.859383e+06,285741.746096,True,1106
1,2025-09-11 00:00:00+00:00,28,0.725587,0.062862,473530.815748,0.155180,5.054174,2.076768e+06,309191.653978,True,1118
2,2025-09-12 00:00:00+00:00,28,0.570358,0.045801,284892.710847,0.061045,2.950956,1.338490e+06,332466.422414,True,1132
3,2025-09-13 00:00:00+00:00,28,0.603151,0.051015,183342.311688,0.130231,3.909055,5.160042e+05,333223.164942,True,1146
4,2025-09-14 00:00:00+00:00,28,0.536549,0.041661,177019.495114,0.058416,2.994198,1.396225e+06,333237.173977,True,1160


=== Worst pools (head) ===


,pool_id,n_days,APY_MAE,APY_MAPE,TVL_MAE,TVL_MAPE,MAX_AE,group_col
39,eaa20dc0-feca-4ef0-8c2d-fc73a6c4a60b,9,98.038288,0.361786,1.100284e+06,0.337688,4.852959e+06,high
35,cee45e20-83e6-42e2-969f-8faef1295778,9,97.256861,0.365954,1.520431e+06,0.512164,8.765646e+06,high
41,f8b4647d-d022-455d-8ed3-5cf87f21f298,60,6.631725,0.345602,3.621841e+05,0.235629,2.802305e+06,low
2,0aeae5ff-b988-4127-b682-2910e3950d41,60,5.241844,0.308873,5.640608e+05,0.200902,4.499510e+06,medium
12,55d0e5ff-fb0f-425e-9398-412e1100fe82,60,1.812928,0.101915,3.049776e+05,0.584516,2.411125e+06,high


In [26]:
bt_instab_daily

,asof,instability_group,n_pools,APY_MAE,APY_MAPE,TVL_MAE,TVL_MAPE,MAX_AE_APY,MAX_AE_TVL
0,2025-09-10 00:00:00+00:00,high,3,4.928201,0.238291,1.365845e+05,0.037217,7.035720,3.898094e+05
1,2025-09-10 00:00:00+00:00,low,15,1.169180,0.147914,2.932808e+05,0.088602,5.988968,1.859383e+06
2,2025-09-10 00:00:00+00:00,medium,6,0.642437,0.060470,1.243394e+05,0.019038,1.272173,3.629208e+05
3,2025-09-11 00:00:00+00:00,high,3,2.685890,0.125685,7.252299e+05,0.178628,5.054174,2.076768e+06
4,2025-09-11 00:00:00+00:00,low,18,0.461943,0.045548,2.552501e+05,0.170221,2.083919,1.153878e+06
...,...,...,...,...,...,...,...,...,...
161,2025-11-07 00:00:00+00:00,low,8,0.584673,0.061412,2.973073e+05,0.067017,1.903892,1.271645e+06
162,2025-11-07 00:00:00+00:00,medium,20,0.794479,0.085103,1.310476e+06,0.071805,2.545072,1.406123e+07
163,2025-11-08 00:00:00+00:00,high,14,4.748167,0.829612,3.406170e+05,0.053353,27.304781,2.523679e+06
164,2025-11-08 00:00:00+00:00,low,10,0.560900,0.077521,6.613318e+04,0.137679,2.732105,2.188983e+05


In [33]:
bt_pool_refit["pred_error_apy"] = (
    bt_pool_refit["pred_global_apy"] - bt_pool_refit["target_apy_t1"]
)

In [35]:
bt_pool_refit["error_direction"] = np.where(
    bt_pool_refit["pred_error_apy"] > 0,  1,
    np.where(bt_pool_refit["pred_error_apy"] < 0, -1, 0)
)

In [ ]:
abs_err = bt_pool_refit["pred_error_apy"].abs()

In [49]:
bt_pool_refit["target_date"] = bt_pool_refit["asof"] + pd.Timedelta(days=1)

In [51]:
bt_pool_refit = bt_pool_refit.sort_values(["pool_id", "target_date"])

In [52]:
bt_pool_refit["actual_apy_prev_day_asof"] = (
    bt_pool_refit
    .groupby("pool_id")["target_apy_t1"]
    .shift(1)          # actual APY of day d-1
)

In [56]:
bt_pool_refit["pred_change_apy"] = (
    bt_pool_refit["pred_global_apy"] - bt_pool_refit["actual_apy_prev_day_asof"]
)

In [58]:
abs_change = bt_pool_refit["pred_change_apy"].abs()
bt_pool_refit["pred_next_day_instability_group"] = pd.cut(
    abs_change,
    bins=[0, 1, 3, np.inf],
    labels=["low", "medium", "high"],
    include_lowest=True
)

In [63]:
import pandas as pd

# ---------- 1. Description / README sheet ----------
readme_df = pd.DataFrame([
    {
        "sheet_name": "daily_metrics",
        "description": "One row per day with global APY/TVL errors and training info (n_pools, MAE, MAPE, stacking flag, etc.)."
    },
    {
        "sheet_name": "pool_day_metrics",
        "description": "One row per (day, pool) with predictions vs realized APY/TVL, errors, cold_start_flag, and instability_group."
    },
    {
        "sheet_name": "overall_summary",
        "description": "Single-row summary over the whole backtest: overall MAE/MAPE for APY & TVL and max errors."
    },
    {
        "sheet_name": "by_pool_summary",
        "description": "One row per pool with aggregated metrics across the backtest (mean errors, max error, instability group)."
    },
    {
        "sheet_name": "instab_daily",
        "description": "One row per (day, instability_group) with APY/TVL error metrics to compare stability regimes."
    },
])

# ---------- 2. Write all outputs to one Excel file ----------
output_path = "backtest_outputs_updated.xlsx"  # change name/path if you want

def _make_excel_safe(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    # Find all timezone-aware datetime columns and drop tz info
    for c in df.select_dtypes(include=["datetimetz"]).columns:
        df[c] = df[c].dt.tz_localize(None)
    return df

bt_daily_x   = _make_excel_safe(bt_daily_refit)
bt_pool_x    = _make_excel_safe(bt_pool_refit)
overall_x    = _make_excel_safe(overall_refit)
by_pool_x    = _make_excel_safe(by_pool_refit)
bt_instab_x  = _make_excel_safe(bt_instab_daily)

with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:
    readme_df.to_excel(writer, sheet_name="README", index=False)

    bt_daily_x.to_excel(writer,  sheet_name="daily_metrics",    index=False)
    bt_pool_x.to_excel(writer,   sheet_name="pool_day_metrics", index=False)
    overall_x.to_excel(writer,   sheet_name="overall_summary",  index=False)
    by_pool_x.to_excel(writer,   sheet_name="by_pool_summary",  index=False)
    bt_instab_x.to_excel(writer, sheet_name="instab_daily",     index=False)

print(f"✅ Excel file written to: {output_path}")

✅ Excel file written to: backtest_outputs_updated.xlsx


In [62]:
import mlflow, os, json, datetime as dt

# Local folder tracking (creates ./mlruns)
mlflow.set_tracking_uri("file:./mlruns")
mlflow.set_experiment("DeFi_Global_LGBM") 


<Experiment: artifact_location='file:///Users/panayotispapoutsis/Documents/StablecoinPoolOpt/mlruns/598204408944768895', creation_time=1762775686470, experiment_id='598204408944768895', last_update_time=1762775686470, lifecycle_stage='active', name='DeFi_Global_LGBM', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [63]:
# Local folder tracking (creates ./mlruns)
mlflow.set_tracking_uri("file:./mlruns")
mlflow.set_experiment("DeFi_Global_LGBM")

run_name = f"global_lgbm_refit__{END_DATE.strftime('%Y%m%d')}"
tags = {
    "project": "stablecoin_yield",
    "notebook": "global_lgbm_v_base.ipynb",
    "stacking": "tvl->apy" if 'USE_TVL_STACKING' in globals() and USE_TVL_STACKING else "off",
    "group_col": GROUP_COL,
}

with mlflow.start_run(run_name=run_name, tags=tags):

    # ---- Params (config + model) ----
    mlflow.log_param("hist_days_panel", HIST_DAYS_PANEL)
    mlflow.log_param("group_col", GROUP_COL)
    mlflow.log_param("train_window_days", TRAIN_WINDOW_DAYS if 'TRAIN_WINDOW_DAYS' in globals() else 60)
    mlflow.log_param("use_tvl_stacking", bool(USE_TVL_STACKING) if 'USE_TVL_STACKING' in globals() else False)

    # feature counts from the *last* fit in the loop (if available)
    mlflow.log_param("apy_feature_count", len(APY_FEATURE_COLS) if 'APY_FEATURE_COLS' in globals() else None)
    mlflow.log_param("tvl_feature_count", len(TVL_FEATURE_COLS) if 'TVL_FEATURE_COLS' in globals() else None)

    # backtest coverage
    mlflow.log_param("bt_start", str(bt_daily_refit["date"].min().date()) if len(bt_daily_refit) else None)
    mlflow.log_param("bt_end",   str(bt_daily_refit["date"].max().date()) if len(bt_daily_refit) else None)
    mlflow.log_param("n_bt_days", int(len(bt_daily_refit)))
    mlflow.log_param("n_bt_obs",  int(len(bt_pool_refit)) if 'bt_pool_refit' in globals() else 0)
    mlflow.log_param("n_unique_pools", int(bt_pool_refit["pool_id"].nunique()) if 'bt_pool_refit' in globals() and len(bt_pool_refit) else 0)

    # Optional: TVL OOF CV score (use mean of per-day OOF if present)
    if 'bt_daily_refit' in globals() and 'tvl_oof_mae_train' in bt_daily_refit.columns:
        tvl_oof_mae = float(bt_daily_refit['tvl_oof_mae_train'].dropna().mean()) if bt_daily_refit['tvl_oof_mae_train'].notna().any() else np.nan
        mlflow.log_metric("tvl_oof_mae", tvl_oof_mae)

    # ---- Overall metrics (use *_refit frames) ----
    mlflow.log_metric("APY_MAE_overall",  float(overall_refit["APY_MAE"].iloc[0]))
    mlflow.log_metric("APY_MAPE_overall", float(overall_refit["APY_MAPE"].iloc[0]))
    mlflow.log_metric("TVL_MAE_overall",  float(overall_refit["TVL_MAE"].iloc[0]))
    mlflow.log_metric("TVL_MAPE_overall", float(overall_refit["TVL_MAPE"].iloc[0]))

    # ---- Daily metrics (step = day index) ----
    for i, row in enumerate(bt_daily_refit.sort_values("date").itertuples(index=False)):
        mlflow.log_metric("APY_MAE_daily",  float(row.APY_MAE),  step=i)
        mlflow.log_metric("APY_MAPE_daily", float(row.APY_MAPE), step=i)
        mlflow.log_metric("TVL_MAE_daily",  float(row.TVL_MAE),  step=i)
        mlflow.log_metric("TVL_MAPE_daily", float(row.TVL_MAPE), step=i)

    # ---- Save artifacts (CSVs + plots) ----
    os.makedirs("mlflow_artifacts", exist_ok=True)
    daily_csv  = "mlflow_artifacts/backtest_daily_metrics_refit.csv"
    pools_csv  = "mlflow_artifacts/backtest_pool_day_metrics_refit.csv"
    bypool_csv = "mlflow_artifacts/backtest_by_pool_metrics_refit.csv"
    overall_csv= "mlflow_artifacts/backtest_overall_refit.csv"

    bt_daily_refit.to_csv(daily_csv, index=False)
    bt_pool_refit.to_csv(pools_csv, index=False)       # fixed: use bt_pool_refit here
    by_pool_refit.to_csv(bypool_csv, index=False)
    overall_refit.to_csv(overall_csv, index=False)

    mlflow.log_artifact(daily_csv)
    mlflow.log_artifact(pools_csv)
    mlflow.log_artifact(bypool_csv)
    mlflow.log_artifact(overall_csv)

    # ---- Plots as artifacts ----
    import matplotlib.pyplot as plt

    # 1) Daily APY performance
    plt.figure(figsize=(10,4))
    plt.plot(bt_daily_refit["date"], bt_daily_refit["APY_MAE"], label="MAE")
    plt.plot(bt_daily_refit["date"], bt_daily_refit["APY_MAPE"], label="MAPE")
    plt.title("Daily APY Performance (refit)")
    plt.xlabel("Date"); plt.ylabel("Error"); plt.legend(); plt.tight_layout()
    apy_png = "mlflow_artifacts/daily_apy_refit.png"
    plt.savefig(apy_png); plt.close()
    mlflow.log_artifact(apy_png)

    # 2) Daily TVL performance
    plt.figure(figsize=(10,4))
    plt.plot(bt_daily_refit["date"], bt_daily_refit["TVL_MAE"], label="MAE")
    plt.plot(bt_daily_refit["date"], bt_daily_refit["TVL_MAPE"], label="MAPE")
    plt.title("Daily TVL Performance (refit)")
    plt.xlabel("Date"); plt.ylabel("Error"); plt.legend(); plt.tight_layout()
    tvl_png = "mlflow_artifacts/daily_tvl_refit.png"
    plt.savefig(tvl_png); plt.close()
    mlflow.log_artifact(tvl_png)

    # ---- Log (last-day) models if present ----
    import joblib
    if 'apy_model' in globals():
        joblib.dump(apy_model, "mlflow_artifacts/model_apy_lgbm_refit.pkl")
        mlflow.log_artifact("mlflow_artifacts/model_apy_lgbm_refit.pkl")
    if 'tvl_model' in globals() and tvl_model is not None:
        joblib.dump(tvl_model, "mlflow_artifacts/model_tvl_lgbm_refit.pkl")
        mlflow.log_artifact("mlflow_artifacts/model_tvl_lgbm_refit.pkl")

    # ---- Save run config as JSON ----
    cfg = {
        "END_DATE": str(END_DATE.date()),
        "HIST_DAYS_PANEL": HIST_DAYS_PANEL,
        "GROUP_COL": GROUP_COL,
        "TRAIN_WINDOW_DAYS": TRAIN_WINDOW_DAYS if 'TRAIN_WINDOW_DAYS' in globals() else 60,
        "USE_TVL_STACKING": bool(USE_TVL_STACKING) if 'USE_TVL_STACKING' in globals() else False,
        "features_APY": APY_FEATURE_COLS if 'APY_FEATURE_COLS' in globals() else [],
        "features_TVL": TVL_FEATURE_COLS if 'TVL_FEATURE_COLS' in globals() else [],
        "n_pools": int(bt_pool_refit["pool_id"].nunique()) if 'bt_pool_refit' in globals() and len(bt_pool_refit) else 0
    }
    with open("mlflow_artifacts/run_config_refit.json", "w") as f:
        json.dump(cfg, f, indent=2)
    mlflow.log_artifact("mlflow_artifacts/run_config_refit.json")

print("✅ MLflow run logged for the daily-refit backtest. Now run:  mlflow ui  and open http://127.0.0.1:5000")

✅ MLflow run logged for the daily-refit backtest. Now run:  mlflow ui  and open http://127.0.0.1:5000
